# 2025 COMP90042 Project

# 1.Dataset Processing

# 2. Model Implementation

In [1]:
import json
import os
import random
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, InputExample, evaluation

class EvidenceClaimRetriever:
    def __init__(self, model_name='all-mpnet-base-v2', batch_size=16, num_epochs=3):
        self.model = SentenceTransformer(model_name)
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.evidence_map = {}
        self.label_mapping = {
            'SUPPORTS': 1,
            'REFUTES': 1,
            'DISPUTED': 1,
            'NOT_ENOUGH_INFO': 0
        }

    def load_data(self, claims_path, evidence_path):
        """Load training data and evidence corpus"""
        # Load evidence corpus
        with open(evidence_path) as f:
            self.evidence_map = json.load(f)

        # Load training claims
        with open(claims_path) as f:
            train_claims = json.load(f)

        # Create training examples
        self.train_examples = []
        for claim_id, claim_data in train_claims.items():
            self._process_claim(claim_data)

    def _process_claim(self, claim_data):
        """Create training examples for a single claim"""
        claim_text = claim_data['claim_text']
        evidence_ids = claim_data['evidences']

        # Add positive examples
        for eid in evidence_ids:
            if eid in self.evidence_map:
                self.train_examples.append(InputExample(
                    texts=[claim_text, self.evidence_map[eid]],
                    label=self.label_mapping[claim_data['claim_label']]
                ))

        # Add negative examples
        self._add_negative_examples(claim_text, evidence_ids, claim_data['claim_label'])

    def _add_negative_examples(self, claim_text, positive_ids, label):
        """Add negative examples with proper ratio"""
        all_evidence_ids = list(self.evidence_map.keys())
        negative_candidates = list(set(all_evidence_ids) - set(positive_ids))

        # Determine number of negatives based on label
        if label == 'NOT_ENOUGH_INFO':
            num_negatives = min(2, len(negative_candidates))
        else:
            num_negatives = min(3 * len(positive_ids), len(negative_candidates))

        # Add selected negatives
        for eid in random.sample(negative_candidates, num_negatives):
            self.train_examples.append(InputExample(
                texts=[claim_text, self.evidence_map[eid]],
                label=0
            ))

    def load_dev_data(self, dev_claims_path):
        """Load separate dev set for validation"""
        with open(dev_claims_path) as f:
            dev_claims = json.load(f)

        # Prepare for InformationRetrievalEvaluator
        self.dev_queries = {}
        self.relevant_docs = {}

        for claim_id, claim_data in dev_claims.items():
            claim_text = claim_data['claim_text']
            evidence_ids = claim_data['evidences']

            self.dev_queries[claim_id] = claim_text
            self.relevant_docs[claim_id] = set(evidence_ids)

    def train(self, output_dir='./trained_model'):
        """Train using dedicated dev set for validation"""
        # Create evaluator using separate dev set
        dev_evaluator = evaluation.InformationRetrievalEvaluator(
            queries=self.dev_queries,
            corpus=self.evidence_map,
            relevant_docs=self.relevant_docs,
            show_progress_bar=True
        )

        # Create DataLoader directly from InputExamples
        train_dataloader = DataLoader(
            self.train_examples,  # Use the list of InputExamples directly
            shuffle=True,
            batch_size=self.batch_size
        )

        # Use MultipleNegativesRankingLoss
        train_loss = losses.MultipleNegativesRankingLoss(self.model)

        # Training configuration
        self.model.fit(
            train_objectives=[(train_dataloader, train_loss)],
            evaluator=dev_evaluator,
            epochs=self.num_epochs,
            evaluation_steps=100,
            warmup_steps=int(len(self.train_examples) * 0.1),
            output_path=output_dir,
            show_progress_bar=True
        )
        
    def retrieve(self, claim_text, top_k=5):
        """Retrieve evidence for a claim"""
        claim_embed = self.model.encode(claim_text)
        scores = torch.nn.functional.cosine_similarity(
            torch.tensor(claim_embed),
            torch.tensor(self.evidence_embeddings)
        )
        top_indices = torch.topk(scores, k=top_k).indices.tolist()
        return [(self.evidence_ids[i], scores[i].item()) for i in top_indices]

    def index_evidence(self, save_path='data/evidence_embeddings.pth'):
        """Precompute all evidence embeddings and optionally save them"""
        self.evidence_ids = list(self.evidence_map.keys())
        self.evidence_texts = [self.evidence_map[eid] for eid in self.evidence_ids]

        # Check if embeddings already exist
        if save_path and os.path.exists(save_path):
            print("Loading precomputed evidence embeddings...")
            data = torch.load(save_path, weights_only=False)
            self.evidence_embeddings = data['embeddings']
            self.evidence_ids = data['ids']
        else:
            print("Computing evidence embeddings...")
            self.evidence_embeddings = self.model.encode(
                self.evidence_texts,
                show_progress_bar=True,
                batch_size=32
            )
            # Save embeddings if save_path is provided
            if save_path:
                print("Saving evidence embeddings...")
                torch.save({'embeddings': self.evidence_embeddings, 'ids': self.evidence_ids}, save_path)


    

In [ ]:
# evidenceRetriever = EvidenceClaimRetriever(
#         model_name='all-MiniLM-L6-v2',
#         batch_size=32,
#         num_epochs=5
#     )


evidenceRetriever = EvidenceClaimRetriever(
        model_name='./climate_retriever')


In [3]:

# 1. Load training data and evidence
evidenceRetriever.load_data(
    claims_path='data/train-claims.json',
    evidence_path='data/evidence.json'
)


In [4]:

# 2. Load separate dev set
evidenceRetriever.load_dev_data('data/dev-claims-baseline.json')

In [5]:

# 3. Precompute evidence embeddings
evidenceRetriever.index_evidence()

Loading precomputed evidence embeddings...


In [6]:


# 4. Train with dev set validation
# evidenceRetriever.train(output_dir='./climate_retriever')


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/Users/jeffphan8816/Desktop/StudyUnimelb/NLP/NLP_a3_gp40/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Cosine Accuracy@1,Cosine Accuracy@3,Cosine Accuracy@5,Cosine Accuracy@10,Cosine Precision@1,Cosine Precision@3,Cosine Precision@5,Cosine Precision@10,Cosine Recall@1,Cosine Recall@3,Cosine Recall@5,Cosine Recall@10,Cosine Ndcg@10,Cosine Mrr@10,Cosine Map@100
100,No log,No log,0.129870,0.220779,0.240260,0.318182,0.129870,0.084416,0.061039,0.044156,0.021645,0.042208,0.050866,0.073593,0.081314,0.185913,0.048753
200,No log,No log,0.116883,0.162338,0.214286,0.272727,0.116883,0.064935,0.054545,0.037662,0.019481,0.032468,0.045455,0.062771,0.068896,0.155878,0.039807
300,No log,No log,0.116883,0.188312,0.201299,0.279221,0.116883,0.075758,0.050649,0.037013,0.019481,0.037879,0.042208,0.061688,0.069796,0.161474,0.040461
359,No log,No log,0.123377,0.181818,0.220779,0.279221,0.123377,0.073593,0.054545,0.036364,0.020563,0.036797,0.045455,0.060606,0.070229,0.165961,0.041159
400,No log,No log,0.123377,0.188312,0.227273,0.292208,0.123377,0.075758,0.054545,0.040909,0.020563,0.037879,0.045455,0.068182,0.075287,0.170127,0.042715
500,3.761600,No log,0.110390,0.201299,0.246753,0.318182,0.110390,0.080087,0.062338,0.041558,0.018398,0.040043,0.051948,0.069264,0.075295,0.169125,0.042775
600,3.761600,No log,0.116883,0.181818,0.253247,0.337662,0.116883,0.075758,0.064935,0.046104,0.019481,0.037879,0.054113,0.076840,0.080029,0.171228,0.044970
700,3.761600,No log,0.123377,0.201299,0.266234,0.344156,0.123377,0.080087,0.067532,0.049351,0.020563,0.040043,0.056277,0.082251,0.084536,0.178700,0.047683
718,3.761600,No log,0.116883,0.181818,0.253247,0.337662,0.116883,0.073593,0.064935,0.048052,0.019481,0.036797,0.054113,0.080087,0.082346,0.173444,0.047236
800,3.761600,No log,0.110390,0.188312,0.266234,0.344156,0.110390,0.080087,0.070130,0.048052,0.018398,0.040043,0.058442,0.080087,0.082196,0.170637,0.048091


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Corpus Chunks:   0%|          | 0/25 [00:00<?, ?it/s]

...

Corpus Chunks:  92%|█████████▏| 23/25 [07:58<00:41, 20.85s/it]

Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Corpus Chunks:  96%|█████████▌| 24/25 [08:19<00:20, 20.87s/it]

Batches:   0%|          | 0/276 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 25/25 [08:22<00:00, 20.11s/it]


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re
from typing import List, Tuple, Dict
import json
from tqdm import tqdm

class LLaMAFilter:
    def __init__(self, model_name="meta-llama/Llama-3.1-8B-Instruct"):
        self.device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
        
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            padding_side="left",
            truncation_side="left"
        )
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
        self.rating_template = (
            "[INST] Rate claim-evidence relevance from 1-5 (1=irrelevant, 5=perfect). "
            "ONLY respond with the number.\n\n"
            "Claim: {claim}\nEvidence: {evidence}\nRating: [/INST]"
        )

    def filter_evidence(self, claim: str, evidence_candidates: List[Tuple[str, str]], final_k: int = 5) -> List[Tuple[str, str, int]]:
        """Optimized evidence filtering with batch processing"""
        # Batch prompts
        prompts = [
            self.rating_template.format(claim=claim, evidence=text)
            for eid, text in evidence_candidates
        ]
        
        # Batch tokenize
        inputs = self.tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=2, 
                temperature=0.1,  
                pad_token_id=self.tokenizer.eos_token_id,
                use_cache=True
            )
        
        scored_evidence = []
        decoded = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
        for (eid, text), output in zip(evidence_candidates, decoded):
            rating = 1  # Default if parsing fails
            if match := re.search(r'\d', output.split("Rating:")[-1]):
                rating = min(5, max(1, int(match.group())))
                
            scored_evidence.append((eid, text, rating))
        
        # Return top-k
        return sorted(scored_evidence, key=lambda x: -x[2])[:final_k]

In [7]:
def get_evidence_text(evidences):
    return [(eid, evidenceRetriever.evidence_map[eid]) for eid in evidences]

In [8]:
def initial_retrieval(retriever, test_claims_path):
    with open(test_claims_path) as f:
        test_claims = json.load(f)
    
    output = {}
    for claim_id, claim_data in test_claims.items():
        # Retrieve candidates
        candidates = retriever.retrieve(claim_data['claim_text'], top_k=10)
        retrieved = [eid for eid, _  in candidates[:10]]
        output[claim_id] = {
            "initial_evidences": retrieved,
            "claim_text": claim_data['claim_text']
        }

    # store output
    with open('data/initial_retrieval_output.json', 'w') as f:
        json.dump(output, f, indent=2)
    return output


In [9]:
res = initial_retrieval(evidenceRetriever, test_claims_path='data/dev-claims-baseline.json')

In [10]:
llama_filter = LLaMAFilter()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [12]:
def process_claims(llama_filterer, dev_claims_set):
    for claim_id, claim_data in dev_claims_set.items():
        candidates = claim_data['initial_evidences']
        filtered = llama_filterer.filter_evidence(claim_data['claim_text'], get_evidence_text(candidates), final_k=6)
        print(filtered)
        dev_claims_set[claim_id]['filtered_evidences'] = [eid for eid, _, _ in filtered]
        # store output
    with open('data/filtered_retrieval_output.json', 'w') as f:
        json.dump(dev_claims_set, f, indent=2)
    return dev_claims_set

res = process_claims(llama_filter,res)
res

[('evidence-572512', '"South Australia has the highest power prices in the world".', 5), ('evidence-67732', '[citation needed] South Australia has the highest retail price for electricity in the country.', 1), ('evidence-780332', 'Industrialised countries such as Canada, the US, and Australia are among the highest per capita consumers of electricity in the world, which is possible thanks to a widespread electrical distribution network.', 1), ('evidence-452156', 'Energy in Australia is the production in Australia of energy and electricity, for consumption or export.', 1), ('evidence-48256', 'One of the most powerful power grids in the world supplies power to the state of Queensland, Australia.', 1), ('evidence-808896', 'The class are the first electric trains to operate in South Australia.', 1)]
[('evidence-559290', "Australia’s total greenhouse gas emissions increased by 0.3% in the first six months of the Carbon Tax to December 2012 to 276.5 Mt CO2 equiv, while Australia's gross domes

{'claim-752': {'initial_evidences': ['evidence-67732',
   'evidence-572512',
   'evidence-780332',
   'evidence-452156',
   'evidence-48256',
   'evidence-808896',
   'evidence-1061888',
   'evidence-252686',
   'evidence-723533',
   'evidence-169170'],
  'claim_text': '[South Australia] has the most expensive electricity in the world.',
  'filtered_evidences': ['evidence-572512',
   'evidence-67732',
   'evidence-780332',
   'evidence-452156',
   'evidence-48256',
   'evidence-808896']},
 'claim-375': {'initial_evidences': ['evidence-334443',
   'evidence-357540',
   'evidence-724468',
   'evidence-316768',
   'evidence-342344',
   'evidence-646443',
   'evidence-553922',
   'evidence-559290',
   'evidence-281950',
   'evidence-441151'],
  'claim_text': 'when 3 per cent of total annual global emissions of carbon dioxide are from humans and Australia prod\xaduces 1.3 per cent of this 3 per cent, then no amount of emissions reductio\xadn here will have any effect on global climate.',
  